In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from torch import cuda
import re
import html

device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

/home/charlie/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


# Settings

In [2]:
MAX_LEN = 512
BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased', do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased').to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

# Preprocessing

In [3]:
train = pd.read_csv("../data/preprocessed/fakes_train.csv", index_col=0, encoding='utf-8', engine='python')
validation = pd.read_csv("../data/preprocessed/fakes_validation.csv", index_col=0, encoding='utf-8', engine='python')

sep_token = '<\|reply\|>'

def regex_text(text):
    text = html.unescape(text)
    text = re.sub(r"\#\'", r"'", text)
    text = re.sub(r"\s+$", '', text)    
    return re.findall(sep_token + " (,?.*)", text)

def label_to_list(label):
    if label:
        return [1]
    else:
        return [0]

def clean_dataframe(df):
    df = df[df['text'].str.contains(sep_token)]
    df['text'] = df['text'].apply(regex_text)
    df = df[df['text'].str.len() != 0]
    df['text'] = df['text'].apply(lambda x: x[0])
    df['label'] = df['label'].apply(label_to_list)
    return df

validation = clean_dataframe(validation)
train = clean_dataframe(train)

/tmp/ipykernel_284122/1971825100.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(regex_text)


In [4]:
dataset = dict()
dataset['validation'] = Dataset.from_pandas(validation, preserve_index=False)
dataset['train'] = Dataset.from_pandas(train, preserve_index=False)
datasets = DatasetDict(dataset)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=MAX_LEN)

tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=["text"],
    )

# Training

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="../output/bert_discriminator",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    save_steps=10000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

trainer.train()

/home/charlie/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 500/30823 [03:04<2:46:42,  3.03it/s]

{'loss': 0.4571, 'learning_rate': 1.967556694676054e-05, 'epoch': 0.02}


  3%|▎         | 1000/30823 [06:03<2:39:22,  3.12it/s]

{'loss': 0.4036, 'learning_rate': 1.9351133893521072e-05, 'epoch': 0.03}


  5%|▍         | 1500/30823 [09:03<2:15:17,  3.61it/s]

...

 99%|█████████▉| 30500/30823 [2:27:21<01:30,  3.58it/s]

{'loss': 0.2871, 'learning_rate': 2.0958375239269377e-07, 'epoch': 0.99}


                                                       
100%|██████████| 30823/30823 [2:32:50<00:00,  3.36it/s]

{'eval_loss': 0.24188634753227234, 'eval_runtime': 236.7721, 'eval_samples_per_second': 57.95, 'eval_steps_per_second': 14.491, 'epoch': 1.0}
{'train_runtime': 9170.304, 'train_samples_per_second': 13.444, 'train_steps_per_second': 3.361, 'train_loss': 0.27492766681514336, 'epoch': 1.0}


TrainOutput(global_step=30823, training_loss=0.27492766681514336, metrics={'train_runtime': 9170.304, 'train_samples_per_second': 13.444, 'train_steps_per_second': 3.361, 'train_loss': 0.27492766681514336, 'epoch': 1.0})

In [8]:
#trainer.save_model('../output/bert_discriminator/final')

In [61]:
import torch
test_input = tokenizer(validation['text'][13871], return_tensors='pt').to(device)
with torch.no_grad():
    logits = model(**test_input).logits

predicted_class_id = logits.argmax().item()
predicted_class_id

1

In [48]:
validation

,text,label
0,"> Andy could come out first, if he hasn't alre...",[1]
1,[deleted] �� ��,[1]
2,[deleted],[1]
3,"My writing is good, and I feel like I have to ...",[1]
4,I read it and liked it.,[1]
...,...,...
13869,It has already been taught. This was recorded ...,[0]
13870,That's exactly what you're saying. You're actu...,[1]
13871,I don't care what other people think of Scalzi...,[1]
13872,I was thinking of it in a similar vein. I thin...,[1]
